In [ ]:
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2
import torchvision.transforms as transforms
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import sys
import os


from data_loader import *
from lib.object_detection_helper import *
from model.RetinaNetFocalLoss import RetinaNetFocalLoss
from model.RetinaNet import RetinaNet


fname = 'RetinaNet-ODAEL-2-export.pth'
#NAME OF MODEL USED FOR STAGE 1 CLASSIFICATION

size=256
path = Path('./')
slidedir = 'WSI'
level = 0
files = []

test_slide_filenames = ['f3741e764d39ccc4d114.svs']

print('Test slides are: ',test_slide_filenames)
print('Model being used is: ', fname)



Test slides are:  ['f3741e764d39ccc4d114.svs']
Model being used is:  RetinaNet-ODAEL-2-export.pth


In [ ]:

# Load slides and initialize boxes + anchors on slides

files=list()
test_slides=list()

for filename in test_slide_filenames:
        slide_path = os.path.join(path, slidedir, filename)
        slide = openslide.open_slide(str(slide_path))
        level = 0#slide.level_count - 1
        files.append(SlideContainer(file=slide_path, level=level, width=size, height=size, y=[[], []], annotations=dict()))
        #test_slides.append(idx)



state = torch.load(fname, map_location='cpu')     if defaults.device == torch.device('cpu')     else torch.load(fname)

model = state.pop('model').cuda()
mean = state['data']['normalize']['mean']
std = state['data']['normalize']['std']



anchors = create_anchors(sizes=[(16,16)], ratios=[1], scales=[0.3, 0.375,0.45])
detect_thresh = 0.3
nms_thresh = 0.4
result_boxes = {}
result_regression = {}




In [ ]:
import multiprocessing
from multiprocessing import Queue as mpQueue
from queue import Queue
import queue
import time

jobQueue=mpQueue()
outputQueue=mpQueue()

def getPatchesFromQueue(slide_container, jobQueue, outputQueue):
    x,y=0,0
    try:
        while (True):
            if (outputQueue.qsize()<100):
                x,y = jobQueue.get(timeout=1)
                outputQueue.put((x,y,slide_container.get_patch(x, y) / 255.))
            else:
                time.sleep(0.1)
    except queue.Empty:
        print('One worker died.')
        pass # Timeout happened, exit



def getBatchFromQueue(batchsize=8):
    images = np.zeros((batchsize,3, size,size))
    x = np.zeros(batchsize)
    y = np.zeros(batchsize)
    try:
        bs=0
        for k in range(batchsize):
            x[k],y[k],images_temp = outputQueue.get(timeout=5)
            images[k] = images_temp.transpose((2,0,1))
            bs+=1
        return images,x,y
    except queue.Empty:
        return images[0:bs],x[0:bs],y[0:bs]



# In[8]:

def rescale_box(bboxes, size: Tensor):
    bboxes[:, :2] = bboxes[:, :2] - bboxes[:, 2:] / 2
    bboxes[:, :2] = (bboxes[:, :2] + 1) * size / 2
    bboxes[:, 2:] = bboxes[:, 2:] * size / 2
    bboxes = bboxes.long()
    return bboxes

In [ ]:
# In[9]:


import time
from functools import partial
class timerObj():
    model = 0
    transform = 0
    get = 0
    postproc = 0
    def __str__(self):
        return 'Model: %.2f, Transform: %.2f, Get: %.2f, Postproc: %.2f' % (self.model, self.transform, self.get, self.postproc)

t = timerObj()
batchsize=8

device = torch.device("cuda:0")
model = model.cuda(device)
with torch.no_grad():
    for idx, slide_container in enumerate(files):

        print('Getting overview from ',slide_container.file)
        size = state['data']['tfmargs']['size']
        result_boxes[idx] = []
        result_regression[idx] = []

        print('Processing WSI ...')

        n_Images=0
        for x in tqdm(range(0, slide_container.slide.level_dimensions[level][0] - 1 * size, int(0.9*size)), position=1):
            for y in range(0, slide_container.slide.level_dimensions[level][1] - 1*  size, int(0.9*size)):
                jobQueue.put((x,y))
                n_Images+=1

        # Set up queued image retrieval
        jobs = []
        for i in range(5):
            p = multiprocessing.Process(target=getPatchesFromQueue, args=(slide_container, jobQueue, outputQueue), daemon=True)
            jobs.append(p)
            p.start()


        for kImage in tqdm(range(int(np.ceil(n_Images/batchsize))), desc='Processing %s' % slide_container.file):


                npBatch,xBatch,yBatch = getBatchFromQueue(batchsize=batchsize)
                imageBatch = torch.from_numpy(npBatch.astype(np.float32, copy=False)).cuda()

                patch = imageBatch

                for p in range(patch.shape[0]):
                    patch[p] = transforms.Normalize(mean,std)(patch[p])

                class_pred_batch, bbox_pred_batch, _ = model(
                    patch[:, :, :, :])

                for b in range(patch.shape[0]):
                    x_real = xBatch[b]
                    y_real = yBatch[b]

                    for clas_pred, bbox_pred in zip(class_pred_batch[b][None,:,:], bbox_pred_batch[b][None,:,:],
                                                                            ):
                        modelOutput = process_output(clas_pred, bbox_pred, anchors, detect_thresh)
                        bbox_pred, scores, preds = [modelOutput[x] for x in ['bbox_pred', 'scores', 'preds']]

                        if bbox_pred is not None:
                            to_keep = nms(bbox_pred, scores, nms_thresh)
                            bbox_pred, preds, scores = bbox_pred[to_keep].cpu(), preds[to_keep].cpu(), scores[to_keep].cpu()

                            t_sz = torch.Tensor([size, size])[None].float()

                            bbox_pred = rescale_box(bbox_pred, t_sz)

                            for box, pred, score in zip(bbox_pred, preds, scores):
                                y_box, x_box = box[:2]
                                h, w = box[2:4]

                                result_boxes[idx].append(np.array([x_box + x_real, y_box + y_real,
                                                                                         x_box + x_real + w, y_box + y_real + h,
                                                                                         pred, score]))


        pickle.dump(result_boxes, open(os.path.join("results", "%s-prediction_results_boxes.p" % (fname)), "wb"))


 43%|████▎     | 63/148 [00:00<00:00, 625.09it/s]



Getting overview from  ./WSI/f3741e764d39ccc4d114.svs
Processing WSI ...


Streaming output truncated to the last 5000 lines.

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2019/4533 [06:18<07:47,  5.37it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2020/4533 [06:18<08:02,  5.21it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2021/4533 [06:18<07:41,  5.44it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2022/4533 [06:18<06:44,  6.21it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2023/4533 [06:18<07:16,  5.75it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2024/4533 [06:19<07:39,  5.47it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2025/4533 [06:19<06:49,  6.12it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2026/4533 [06:19<07:23,  5.65it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2027/4533 [06:19<07:21,  5.67it/s]

Processing ./WSI/f3741e764d39ccc4d114.svs:  45%|████▍     | 2028/4533 [06:19<07:51, 

One worker died.
One worker died.
One worker died.
One worker died.
One worker died.




Processing ./WSI/f3741e764d39ccc4d114.svs: 100%|██████████| 4533/4533 [21:05<00:00,  3.58it/s]


**2nd Stage** \\
Uses ResNet model and result boxes from the RetinaNet to output final dictionary of coordinates

In [ ]:
import sys
from lib.processSecondStageClassification import *
import pickle
import os

#Temporarily hard-coded file name for testing
data = 'RetinaNet-ODAEL-2-prediction_results_boxes.p'

data_path =  os.path.join('results', data)
model_path = os.path.join('trained_models', '2nd_stage', 'CellClassifier_128px.pth')
p = pickle.load(open(fname, 'rb'))
#path,fname = os.path.split(fname)

basepath='WSI'
out_2ndstage = processSecondStageClassifier(p,  intermediate=path+'2ndstage_'+fname, modelpath=model_path, basepath=basepath)
pickle.dump(out_2ndstage, open(os.path.join('extracted', path+'2ndstage_'+fname),'wb'))


ModuleNotFoundError: ignored

**Setup**

In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/MC/model/')
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/MC/')
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/MC/lib/')

In [ ]:
!pip install setuptools==42.0.0

     |████████████████████████████████| 583kB 2.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 49.2.0
    Uninstalling setuptools-49.2.0:
      Successfully uninstalled setuptools-49.2.0


In [ ]:
!sudo pip3 install -U SlideRunner

     |████████████████████████████████| 358kB 2.8MB/s 
     |████████████████████████████████| 317kB 12.2MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 76.6MB 118kB/s 
     |████████████████████████████████| 286kB 37.9MB/s 
  Created wheel for openslide-python: filename=openslide_python-1.1.1-cp36-cp36m-linux_x86_64.whl size=27365 sha256=2fbb333d86ec5573b9bf03d3245348ab7c180801096802d56b4f6f59e89e4c99
  Stored in directory: /root/.cache/pip/wheels/56/44/7e/16c9fc72cfbf1bffe48676b6835843d21abcc56566e958e7d6
  Created wheel for rollbar: filename=rollbar-0.15.0-cp36-none-any.whl size=61685 sha256=61fa77d59cec0402daad5b7450164738610b797774d6a567d4869fab3ad83604
  Stored in directory: /root/.cache/pip/wheels/1c/b5/b1/290546a7a87f78d5ce22b428d159bb4a783fb256168f9b552f
Successfully built openslide-python rollbar


In [ ]:
!sudo apt-get install openslide-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (181 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/MC

In [ ]:
%ls

databases/              prediction.ipynb
data_loader.py          __pycache__/
Evaluation.ipynb        README.md
Inference-RetinaNet.py  RetinaNet-CCMCT-HEAEL.ipynb
lib/                    RetinaNet-CCMCT-MEL.ipynb
Mitotic_Figures.ipynb   RetinaNet-CCMCT-ODAEL.ipynb
model/                  Setup.ipynb
models/                 WSI/
